## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram import atprogram

In [4]:
from os import getcwd, path, pardir
import pickle

## Compile and program project

In [5]:
project_path = path.curdir + "/" + "AES-128_Crypto-Accelerator_Countermeasures"
project_name =  "AES-128_Crypto-Accelerator_Countermeasures"
device_name = "ATSAML21J18B"
project_path 

'./AES-128_Crypto-Accelerator_Countermeasures'

In [6]:
atprogram.atprogram(project_path, device_name = device_name ,verbose=3)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:55436
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00001130.
[DEBUG] Memory segment base written at 0x00001130. Size = 0x00000084.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Starting execution of "info"
[ERROR] Error in TCF lockbit format. Missed key Could not get Module LOCKBIT for ATSAML21J18B. (TCF Error code: 131120)
[DEBUG] Command "info" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.
Tool edbg has firmware version: 03.25
Target voltage: 3.31 V

Device information:

Name:       ATSAML21J18B 

0

In [7]:
#atprogram(path.abspath(path.join(*project_path)), verbose=2)

In [8]:
atprogram.get_project_size(project_path, device_name = device_name , verbose=2)

{'text': 4400, 'data': 132, 'bss': 8332, 'dec': 12864, 'hex': 12864, 'filename': 'AES-128_Crypto-Accelerator_Countermeasures.elf'}


{'text': 4400,
 'data': 132,
 'bss': 8332,
 'dec': 12864,
 'hex': 12864,
 'filename': 'AES-128_Crypto-Accelerator_Countermeasures.elf'}

## Data Logging

In [9]:
live_plot = False

Create a figure for the plot.

In [10]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [11]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_aes_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment AES-128 Flash",
}

Stop criteria to pass to the logger:

In [12]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

In [13]:
#with DGILib() as dgilib:
#     dgilib.get_major_version()

Perform the measurement.

In [15]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

In [16]:
data.length()

{48: 6005, 256: 858000}

In [17]:
print(data)

Interfaces:
	  48:   gpio,    samples:    6005
	 256:   power,   samples:  858000



# Store Data

In [18]:
import pickle

In [19]:
pickle.dump(data, open("aes_flash_logger_data.p", "wb"))

# Load Data

In [20]:
data = pickle.load(open("aes_flash_logger_data.p", "rb"))

In [21]:
#iteration = 0
#name = "AES-128_Flash"
#data = pickle.load(open(path.join(path.pardir, path.pardir, f"{name}_{iteration}.p"), "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [22]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [23]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [24]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [25]:
print(len(aes_charge), len(aes_time), len(flash_charge), len(flash_time))
# cutoff = min(len(aes_charge), len(aes_time), len(flash_charge), len(flash_time))
# aes_charge = aes_charge[:cutoff]
# aes_time = aes_time[:cutoff]
# flash_charge = flash_charge[:cutoff]
# flash_time = flash_time[:cutoff]

750 750 750 750


In [26]:
# length = len(aes_charge)
# assert length == len(aes_time)
# assert length == len(flash_charge)
# assert length == len(flash_time)
# print(length)

In [27]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [28]:
aes_flash_write_charge = flash_charge[0::2]
aes_flash_read_charge = flash_charge[1::2]
aes_flash_write_time = flash_time[0::2]
aes_flash_read_time = flash_time[1::2]

In [29]:
len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)

(375, 375, 375, 375, 375, 375, 375, 375)

In [30]:
drop = 0
cutoff = min(len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)) - drop
aes_encrypt_charge = aes_encrypt_charge[:cutoff]
aes_decrypt_charge = aes_decrypt_charge[:cutoff]
aes_encrypt_time = aes_encrypt_time[:cutoff]
aes_decrypt_time = aes_decrypt_time[:cutoff]
aes_flash_write_charge = aes_flash_write_charge[:cutoff]
aes_flash_read_charge = aes_flash_read_charge[:cutoff]
aes_flash_write_time = aes_flash_write_time[:cutoff]
aes_flash_read_time = aes_flash_read_time[:cutoff]

In [31]:
length = len(aes_encrypt_charge)
assert length == len(aes_decrypt_charge)
assert length == len(aes_encrypt_time)
assert length == len(aes_decrypt_time)
assert length == len(aes_flash_write_charge)
assert length == len(aes_flash_read_charge)
assert length == len(aes_flash_write_time)
assert length == len(aes_flash_read_time)
print(length)

375


# Convert to Joule

In [32]:
voltage = 3.31
j_scale = 1e3 # m
t_scale = 1e3 # m
model_j_scale = 1e6 # n
model_t_scale = 1e3 # u

experiment_name =  project_name

In [33]:
aes_encrypt_energy = aes_encrypt_charge[:cutoff]
aes_flash_write_energy = aes_flash_write_charge[:cutoff]
aes_flash_read_energy = aes_flash_read_charge[:cutoff]
aes_decrypt_energy = aes_decrypt_charge[:cutoff]
aes_encrypt_time_s = aes_encrypt_time[:cutoff]
aes_flash_write_time_s = aes_flash_write_time[:cutoff]
aes_flash_read_time_s = aes_flash_read_time[:cutoff]
aes_decrypt_time_s = aes_decrypt_time[:cutoff]

for i in range(len(aes_encrypt_energy)):
    aes_encrypt_energy[i] = aes_encrypt_energy[i] * voltage * j_scale
for i in range(len(aes_flash_write_energy)):
    aes_flash_write_energy[i] = aes_flash_write_energy[i] * voltage * j_scale
for i in range(len(aes_flash_read_energy)):
    aes_flash_read_energy[i] = aes_flash_read_energy[i] * voltage * j_scale
for i in range(len(aes_decrypt_energy)):
    aes_decrypt_energy[i] = aes_decrypt_energy[i] * voltage * j_scale
    
for i in range(len(aes_encrypt_time_s)):
    aes_encrypt_time_s[i] = aes_encrypt_time_s[i] * t_scale
for i in range(len(aes_flash_write_time_s)):
    aes_flash_write_time_s[i] = aes_flash_write_time_s[i] * t_scale
for i in range(len(aes_flash_read_time_s)):
    aes_flash_read_time_s[i] = aes_flash_read_time_s[i] * t_scale
for i in range(len(aes_decrypt_time_s)):
    aes_decrypt_time_s[i] = aes_decrypt_time_s[i] * t_scale

In [34]:
MBEDTLS_AES_BLOCK_SIZE = 16
STEP_SIZE = MBEDTLS_AES_BLOCK_SIZE
MIN_NUM_BYTES = STEP_SIZE
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + STEP_SIZE * len(aes_encrypt_energy), STEP_SIZE)
print(f"MAX_NUM_BYTES: {num_bytes[-1]}")

MAX_NUM_BYTES: 6000


In [35]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)
# pars['intercept'].set(min=0)

In [36]:
results = []
ylabels = (['Energy [mJ]'] * 2 + ['Time [ms]'] * 2) * 2 + ['Energy [mJ]'] + ['Time [ms]']
parameter_names = [
    'Encrypt Energy',
    'Flash Write Energy',
    'Flash Read Energy',
    'Decrypt Energy',
    'Encrypt Time',
    'Flash Write Time',
    'Flash Read Time',
    'Decrypt Time',
    'Total Energy',
    'Total Time',
]
for y in [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s,
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy)],
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s)]]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    fig, grid = result.plot(
        xlabel='Checkpoint Size [Bytes]',
        ylabel=ylabels[len(results)])
    fig.tight_layout(rect=(0.05, 0.05, 1, 1))
    fig.set_size_inches(5, 4.5, forward=True)
    fig.canvas.set_window_title(
        f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    fig.show()
    fig.savefig( f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 375
    # variables        = 2
    chi-square         = 3.3200e-06
    reduced chi-square = 8.9007e-09
    Akaike info crit   = -6949.43011
    Bayesian info crit = -6941.57626
[[Variables]]
    slope:      1.0445e-05 +/- 2.8128e-09 (0.03%) (init = 0)
    intercept:  3.5590e-04 +/- 9.7633e-06 (2.74%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.867
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 375
    # variables        = 2
    chi-square         = 0.02731884
    reduced chi-square = 7.3241e-05
    Akaike info crit   = -3568.66427
    Bayesian info crit = -3560.81042
[[Variables]]
    slope:      1.1693e-04 +/- 2.5515e-07 (0.22%) (init = 0)
    intercept:  0.01240888 +/- 8.8565e-04 (7.14%) (init = 1)
[[Correlations]] (unreport

In [37]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title(f"Analysis {experiment_name}")

In [38]:
charge_color = 'r'
time_color = 'b'

In [39]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [mJ]', color=charge_color)
ax2.set_ylabel('Time [ms]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [40]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_energy, charge_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_write_energy, charge_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_read_energy, charge_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_decrypt_energy, charge_color+'--', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_encrypt_time_s, time_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_write_time_s, time_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_read_time_s, time_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_decrypt_time_s, time_color+'--', label=f'{parameter_names[len(lines)]}')
ax1.legend(handles=lines)
ax1.set_title(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.04} $\mu$s\n")
fig2.tight_layout()
fig2.savefig(f"Analysis {experiment_name}")
fig2.show()

In [41]:
print(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.020} $\mu$s\n"
)

Encrypt Energy: Slope 10.445482046553758337 nJ/B, Intercept 355.89601143204419031 nJ
Flash Write Energy: Slope 116.92978838793817431 nJ/B, Intercept 12408.878903546472429 nJ
Flash Read Energy: Slope 3.2971426877064402561 nJ/B, Intercept -130.49524219344021958 nJ
Decrypt Energy: Slope 10.331846857126599559 nJ/B, Intercept 660.82965050306972898 nJ
Encrypt Time: Slope 8.0891743293535469661 $\mu$s/B, Intercept 182.84343957798944302 $\mu$s
Flash Write Time: Slope 21.795398897259882887 $\mu$s/B, Intercept 2742.5103394111988564 $\mu$s
Flash Read Time: Slope 2.1247504514704900203 $\mu$s/B, Intercept 28.342629188534363749 $\mu$s
Decrypt Time: Slope 7.9832239670443829738 $\mu$s/B, Intercept 401.86904993710493272 $\mu$s
Total Energy: Slope 141.00425997932492805 nJ/B, Intercept 13295.109329833687298 nJ
Total Time: Slope 39.992547663404884872 $\mu$s/B, Intercept 3355.5655430253032137 $\mu$s



In [42]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(aes_encrypt_energy, open("aes_flash_encrypt_energy_mJ.p", "wb"))
pickle.dump(aes_decrypt_energy, open("aes_flash_decrypt_energy_mJ.p", "wb"))
pickle.dump(aes_flash_write_energy, open("aes_flash_write_energy_mJ.p", "wb"))
pickle.dump(aes_flash_read_energy, open("aes_flash_read_energy_mJ.p", "wb"))
pickle.dump(aes_encrypt_time_s, open("aes_flash_encrypt_time_ms.p", "wb"))
pickle.dump(aes_decrypt_time_s, open("aes_flash_decrypt_time_ms.p", "wb"))
pickle.dump(aes_flash_write_time_s, open("aes_flash_write_time_ms.p", "wb"))
pickle.dump(aes_flash_read_time_s, open("aes_flash_read_time_ms.p", "wb"))

In [43]:
aes = [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s]
for i in aes:
    print(len(i), len(i)*16)

375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000


## Write config file

In [44]:
import json

config = {}
config["name"] = "AES-128 Flash"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["AES-128 Encrypt", "AES-128 Decrypt"], 3: ["AES-128 Flash Write", "AES-128 Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["AES-128 Encrypt", "AES-128 Flash Write"],
                                        "load": ["AES-128 Flash Read", "AES-128 Decrypt"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": MBEDTLS_AES_BLOCK_SIZE}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

# Write model data

In [45]:
dump_pickle = True
fit_lm = True
verbose = 2
show_lm_plot = 2
# drop = 1

In [46]:
# Parse data
analysis_config = config.get("analysis")
result_types = analysis_config.get("result_types")
x_step = analysis_config.get("x_step")
parsed_data = {}
for pin, parameter_names in analysis_config.get("pins").items():
    data2 = power_and_time_per_pulse(
        data, int(pin), stop_function=stop_function)
    num_names = len(parameter_names)
    for i, parameter_name in enumerate(parameter_names):
        end_index = -drop * num_names or None
        parsed_data[parameter_name] = {
            result_types[0]: data2[0][i:end_index:num_names],
            result_types[1]: data2[1][i:end_index:num_names],
            "x_step": x_step}
if dump_pickle:
    pickle.dump(parsed_data, open(
        path.join(path.curdir,
                  f"{config_dict.get('file_name_base')}_looped.p"), "wb"))

In [47]:
# Fit lm
if fit_lm:
    model = None
    if model is None:
        def line(x, intercept, slope):
            """a line"""
            return [intercept + slope*i for i in x]

        model = Model(line)
        params = model.make_params(intercept=0, slope=1)
#         params['intercept'].set(min=0)
    else:
        params = model.params

    model_results = {}
    labels = analysis_config.get("labels")
    for parameter_name in parsed_data.keys():
        length = len(parsed_data[parameter_name][result_types[0]])
        x_step = parsed_data[parameter_name]["x_step"]
        num_bytes = range(x_step, (length+1)*x_step, x_step)
        if verbose:
            print(
                f"Fitting model to {parameter_name} with {length} " +
                f"samples, from {min(num_bytes)} to {max(num_bytes)} "
                f"bytes in steps of {x_step}.")
        model_result = {}
        for result_type in result_types:
            model_result[result_type] = model.fit(
                parsed_data[parameter_name][result_type], params,
                x=num_bytes)
            if verbose >= 2:
                print(model_result[result_type].fit_report())
            # Plot multiple view
            if show_lm_plot >= 2:
                fig, grid = model_result[result_type].plot(
                    xlabel=f"{labels[result_type]['x']} " +
                           f"[{labels[result_type]['x_unit']}]",
                    ylabel=f"{labels[result_type]['y']} " +
                           f"[{labels[result_type]['y_unit']}]")
                fig.canvas.set_window_title(
                    f"Residuals of {parameter_name}")
                fig.tight_layout()
                fig.show()
            model_results[parameter_name] = model_result

    # Plot single view
    if show_lm_plot:
        import matplotlib.pyplot as plt
        fig = plt.figure(figsize=(9, 6))
        fig.canvas.set_window_title(f"Analysis {config.get('name')}")
        colors = dict(zip(result_types, ['r', 'b']))
        line_styles = (
            line_style for line_style in ('-', '--', '-.', ':') * 2)
        # fig.suptitle(f"Energy analysis of {config.get('name')}")
        ax = {}
        ax[result_types[0]] = fig.add_subplot(1, 1, 1)
        ax[result_types[1]] = ax[result_types[0]].twinx()
        ax[result_types[0]].set_xlabel(
            f"{labels[result_types[0]]['x']} " +
            f"[{labels[result_types[0]]['x_unit']}]")
        for result_type in result_types:
            ax[result_type].set_ylabel(
                f"{labels[result_type]['y']} " +
                f"[{labels[result_type]['y_unit']}]",
                color=colors[result_type])
            ax[result_type].tick_params('y', colors=colors[result_type])

        lines = []
        title_str = ""
        for parameter_name in parsed_data.keys():
            length = len(parsed_data[parameter_name][result_types[0]])
            x_step = parsed_data[parameter_name]["x_step"]
            num_bytes = range(x_step, (length+1)*x_step, x_step)
            model_result = {}
            line_style = next(line_styles)
            for result_type in result_types:
                label = f"{parameter_name} {labels[result_type]['y']}"
                lines += ax[result_type].plot(
                    num_bytes, parsed_data[parameter_name][result_type],
                    colors[result_type] + line_style, label=label)
                title_str += f"{label} "
                for param in params.keys():
                    title_str += "".join(
                        f"{params[param].name.capitalize()}: ")
                    title_str += "".join(
                        f"{model_results[parameter_name][result_type].params[param].value: .03} ")
                    title_str += "".join(
                        f"{labels[result_type]['y_unit']}, ")
                title_str = title_str[:-2] + \
                    f" per {labels[result_type]['x_unit']}\n"
        ax[result_types[0]].legend(handles=lines)
        ax[result_types[0]].set_title(title_str[:-1])
        # fig.tight_layout()
        fig.tight_layout(rect=(0.05, 0.05, 1, 1))
        fig.set_size_inches(8, 6, forward=True)
        fig.show()

    # Save model results to file
    if dump_pickle:
        model_results_dump = {}
        for parameter_name in model_results.keys():
            model_results_dump[parameter_name] = {}
            for result_type in model_results[parameter_name].keys():
                model_results_dump[parameter_name][result_type] = \
                    model_results[parameter_name][result_type].values

        pickle.dump(model_results_dump, open(path.join(
            path.curdir,
            f"{config_dict.get('file_name_base')}_model.p"), "wb"))

Fitting model to AES-128 Encrypt with 375 samples, from 16 to 6000 bytes in steps of 16.
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 10
    # data points      = 375
    # variables        = 2
    chi-square         = 3.0302e-13
    reduced chi-square = 8.1240e-16
    Akaike info crit   = -13027.9577
    Bayesian info crit = -13020.1039
[[Variables]]
    intercept:  1.0752e-07 +/- 2.9496e-09 (2.74%) (init = 0)
    slope:      3.1557e-09 +/- 8.4979e-13 (0.03%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(intercept, slope) = -0.867
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 375
    # variables        = 2
    chi-square         = 1.6789e-06
    reduced chi-square = 4.5009e-09
    Akaike info crit   = -7205.12275
    Bayesian info crit = -7197.26889
[[Variables]]
    intercept:  1.8284e-04 +/- 6.9428e-06 (3.80%) (init = 

# Total from measurement

In [48]:
n_samples = 5995

total_energy = sum(aes_encrypt_energy[:n_samples]) + sum(aes_flash_write_energy[:n_samples]) + sum(aes_flash_read_energy[:n_samples]) + sum(aes_decrypt_energy[:n_samples])
total_time = sum(aes_encrypt_time_s[:n_samples]) + sum(aes_flash_write_time_s[:n_samples]) + sum(aes_flash_read_time_s[:n_samples]) + sum(aes_decrypt_time_s[:n_samples])
print(total_energy, total_time)

164.03847125616667 46369.93079039993


In [49]:
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy)],
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s)]

[5.015837866666664,
 5.081835733333339,
 5.807812266666643,
 6.269797333333328,
 6.335897599999973,
 6.930448000000033,
 7.062456533333345,
 7.457823466666652,
 8.117671999999992,
 8.24964319999999,
 8.77750346666667,
 9.239477333333301,
 9.570360000000111,
 9.768473599999995,
 10.164202666666622,
 10.559829333333383,
 15.31192053333341,
 15.906642666666581,
 16.43415040000007,
 16.631462400000018,
 16.83060000000003,
 17.358981866666397,
 17.687528000000064,
 18.215675200000224,
 18.67905653333346,
 18.678556266666703,
 19.40290240000009,
 19.733361600000055,
 20.195510400000003,
 20.395483199999838,
 20.922873600000113,
 21.250969599999834,
 26.00518079999992,
 25.939119999999654,
 26.3331487999996,
 26.86351946666643,
 27.126318399999903,
 27.520887999999964,
 28.11486399999974,
 28.249642666666517,
 28.645463466666406,
 29.3693002666664,
 29.43814080000018,
 30.095448533333574,
 30.491689066666886,
 30.822899199999917,
 30.952499200000094,
 31.414712533333276,
 36.037232000000195,
